In [8]:
%load_ext autoreload
%autoreload 2

from os.path import join as oj

import numpy as np

import matplotlib
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation

from IPython.display import HTML

# let James know if you run into permissions errors
# TODO: replace matplotlib.animation with napari
plt.rcParams['animation.ffmpeg_path'] = '/clusterfs/fiona/jpduncan/.conda/envs/neurohub/bin/ffmpeg'

import sys
sys.path.append('../src')
from puncta import *
from util import *

data_dir = '/clusterfs/fiona/yugroup/exllsm'
out_dir = '/clusterfs/fiona/jpduncan/output'
m3 = 'm3_Sample4/slice-tiff'

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
%%time

# m3_ch0 = read_vol(oj(data_dir, m3, 'ch0'), n_slices=100, verbose=True)
m3_ch1 = read_vol(oj(data_dir, m3, 'ch1'), n_slices=100, verbose=True)
# m3_ch2 = read_vol(oj(data_dir, m3, 'ch2'), n_slices=100, verbose=True)

Reading 100 slices from /clusterfs/fiona/yugroup/exllsm/m3_Sample4/slice-tiff/ch1, starting at 0.tif


100%|██████████| 99/99 [00:13<00:00,  7.50it/s]

Volume shape: (100, 4113, 3170)
CPU times: user 6.16 s, sys: 6.82 s, total: 13 s
Wall time: 13.2 s


In [9]:
%%time

# about 6 minutes for 100 slices
ch0_dict = calc_synapse_density(m3_ch0, find_local_max=False, verbose=True, save_filepath=oj(out_dir, 'm3', 'ch0_calc_synapse_density.pkl'))

Otsu's threshold: 11.949
Duration of "apply_threshold" call: 1.46e+02s
Duration of "filter_small_artifacts" call: 1.17e+02s
Duration of "regionprops_table" call: 30.9s
Duration of "KDTree" call: 0.00133s
Duration of "KDTree.query_radius" call: 0.00571s
CPU times: user 4min, sys: 2min 27s, total: 6min 28s
Wall time: 6min 31s


In [4]:
%%time
# about 6 minutes for 100 slices
ch1_dict = calc_synapse_density(m3_ch1, find_local_max=False, verbose=True, save_filepath=oj(out_dir, 'm3', 'ch1_calc_synapse_density.pkl'))

Otsu's threshold: 5.027
Duration of "apply_threshold" call: 1.24e+02s
Duration of "filter_small_artifacts" call: 1.33e+02s
Duration of "regionprops_table" call: 5.81s
Duration of "KDTree" call: 0.00214s
Duration of "KDTree.query_radius" call: 0.0188s
CPU times: user 3min 43s, sys: 2min 9s, total: 5min 52s
Wall time: 5min 54s


In [6]:
mask = ch1_dict['synapse_density_mask']

In [15]:
mask[0,:,:].shape

(4113, 3170)

In [18]:
im_cp = m3_ch1[:, 0:500, 1250:1750].copy()
im_cp[mask[:, 0:500, 1250:1750] != 0] = np.amax(im_cp)

fig, ax = plt.subplots()
l = ax.imshow(im_cp[0,:,:], vmin = 0, vmax = np.amax(im_cp))

def animate(i):
    l.set_data(im_cp[i,:,:])

ani = FuncAnimation(fig, animate, frames=im_cp.shape[0])
plt.close()

HTML(ani.to_html5_video())